In [1]:
import torch
from torch import nn
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi, SNLE, MNLE, SNRE, SNRE_A
from sbi.utils.posterior_ensemble import NeuralPosteriorEnsemble
from sbi.utils import BoxUniform
from sbi.utils import MultipleIndependent
from sbi.neural_nets.embedding_nets import PermutationInvariantEmbedding, FCEmbedding
from sbi.utils.user_input_checks import process_prior, process_simulator
from sbi.utils import get_density_thresholder, RestrictedPrior
from sbi.utils.get_nn_models import posterior_nn
import numpy as np
import moments
from matplotlib import pyplot as plt
import pickle
import os
import seaborn as sns
import datetime
import pandas as pd
import logging
import atexit
import torch.nn.functional as F
import subprocess
import sparselinear as sl
from sortedcontainers import SortedDict
from scipy.spatial import KDTree
import os
import re
from monarch_linear import MonarchLinear
import pdb
logging.getLogger('matplotlib').setLevel(logging.ERROR) # See: https://github.com/matplotlib/matplotlib/issues/14523
from collections import defaultdict
from sbi.analysis import pairplot
from tqdm import tqdm


/home/rahul/PopGen/SimulationSFS/SFS/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/home/rahul/PopGen/SimulationSFS/SFS/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/rahul/PopGen/SimulationSFS/SFS/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [28]:
sample_size=55855
the_device='cuda'


In [29]:
class SummaryNet(nn.Module):
    def __init__(self, sample_size, block_sizes, dropout_rate=0.0):
        super().__init__()
        self.sample_size = sample_size # For monarch this needs to be divisible by the block size
        self.block_size = block_sizes
        self.linear4 = MonarchLinear(sample_size, int(sample_size / 10), nblocks=self.block_size[0]) # 11171
        self.linear5 = MonarchLinear(int(self.sample_size / 10), int(self.sample_size / 10) , nblocks=self.block_size[1]) # 11171
        self.linear6 = MonarchLinear(int(self.sample_size / 10), int(self.sample_size / 10), nblocks=self.block_size[2]) # 11171

        self.model = nn.Sequential(self.linear4, nn.Dropout(dropout_rate), nn.GELU(),
                                   self.linear5, nn.Dropout(dropout_rate), nn.GELU(),
                                   self.linear6) 
    def forward(self, x):
        
        x=self.model(x)
        return x

In [30]:
def get_sim_datafrom_hdf5(path_to_sim_file: str):
    """_summary_

    Args:
        path_to_sim_file (str): _description_
    """    
    #TODO probably will be better to use https://github.com/quantopian/warp_prism for faster look-up tables
    global loaded_file 
    global loaded_file_keys
    global loaded_tree
    import h5py
    loaded_file = h5py.File(path_to_sim_file, 'r')
    loaded_file_keys = list(loaded_file.keys())
    loaded_tree = KDTree(np.asarray(loaded_file_keys)[:,None]) # needs to have a column dimension

In [101]:
def aggregated_generate_sim_data(prior: float) -> torch.float32:

    data = np.zeros((sample_size*2-1))
    theprior = prior
    gammas = -1*10**(theprior.cpu().numpy().squeeze())
    #theprior=prior
    #mis_id = prior[-1].cpu().numpy()
    mis_id=0
    for a_prior in gammas:
        _, idx = loaded_tree.query(a_prior, k=(1,)) # the k sets number of neighbors, while we only want 1, we need to make sure it returns an array that can be indexed
        fs = loaded_file[loaded_file_keys[idx[0]]][:]
        #fs = fs*.01552243512  # scale to lof theta
        fs = fs*.2077370846  # scale to missense theta
        fs = (1 - mis_id)*fs + mis_id * fs[::-1]
        data += fs 
    data /= theprior.shape[0]
    return torch.exp(torch.log(torch.nn.functional.relu(torch.tensor(data, device=the_device)+1)).type(torch.float32))

In [32]:
def load_true_data(a_path: str, type: int) -> torch.float32:
    """Loads a true SFS, note that the sample size must be consistent with the passed parameters

    Args:
        path (str): Where the true-SFS is located, must be a numpy array
        type (int): is data stored in numpy pickle (0) or torch pickle (1)

    Returns:
        Returns the SFS of the true data-set
    """
    if type == 0:
        sfs = np.load(a_path)
        sfs = torch.tensor(sfs, device=the_device).type(torch.float32)
    else:
        sfs = torch.load(a_path)
        sfs.to(the_device)
    assert sfs.shape[0] == sample_size*2-1, "Sample Size must be the same dimensions as the Site Frequency Spectrum, SFS shape: {} and sample shape (2*N-1): {}".format(sfs.shape[0], sample_size*2-1)

    return sfs 

In [7]:
get_sim_datafrom_hdf5('chr10_sim_genome_wide_mut_sfs.h5')

In [12]:
udist = torch.distributions.uniform.Uniform(-6.0*torch.ones(50),4.0*torch.ones(50))
uniform_samples = udist.sample((1000,))

In [102]:
last_posterior_lof=torch.load('nfe_restriction_classifier_lof_embedding_genome_wide_fixed_theta.pkl')
last_posterior_missense=torch.load('nfe_restriction_classifier_mis_embedding_genome_wide.pkl')


In [103]:
true_x_mis = (load_true_data('emperical_missense_sfs_nfe.npy', 0)[:-1]).unsqueeze(0)
true_x_lof = (load_true_data('emperical_lof_sfs_nfe.npy', 0)[:-1]).unsqueeze(0)


In [10]:
# If using a posterior-like proposal
accept_reject_fn = get_density_thresholder(last_posterior, quantile=1e-5, num_samples_to_estimate_support=100000)
proposal = RestrictedPrior(last_posterior._prior, accept_reject_fn, last_posterior, sample_with="sir", device='cuda')

In [104]:
# If you using a classifier based proposal
proposal_lof = last_posterior_lof.restrict_prior(allowed_false_negatives=0.0)
proposal_mis = last_posterior_missense.restrict_prior(allowed_false_negatives=0.0)

In [36]:
predicted_fs=[]


In [105]:
obs_samples_lof = proposal_lof.sample((1000,), oversampling_factor=1024)
obs_samples_mis = proposal_mis.sample((1000,), oversampling_factor=1024)


In [107]:
print("Shape of sampled selection coefficients: {}".format(obs_samples_mis.shape))

Shape of sampled selection coefficients: torch.Size([1000, 60])


In [42]:
predicted_fs=[]

In [43]:
for obs_sample in tqdm(obs_samples):
    fs = aggregated_generate_sim_data(obs_sample)
    predicted_fs.append(fs.unsqueeze(0).cpu().numpy())


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:38<00:00, 25.86it/s]


In [44]:
new_predicted_fs = np.asarray(predicted_fs).squeeze(1)
print("Shape of frequency spectrum containing all bins {}".format(new_predicted_fs.shape))

Shape of frequency spectrum containing all bins (1000, 111709)


In [196]:
plt.ioff()
plt.tight_layout()

fig = plt.figure(figsize=(30,30))
fig.subplots_adjust(hspace=0.6, wspace=0.6)
for i in range(1, 25):
    ax = fig.add_subplot(5, 5, i)
    sns.histplot(np.log10(new_predicted_fs[:,i-1]))
    plt.axvline(x=np.mean(np.log10(new_predicted_fs[:,i-1])), color='m', label="mean")
    plt.axvline(x=np.median(np.log10(new_predicted_fs[:,i-1])), color='k', label="median")
    ax.axline((np.log10(true_x[i-1]), 1), (np.log10(true_x[i-1]),100), marker='+', c='r', label="Emperical SFS")
    plt.title("Emperical SFS: 10^{:.3f} at bin: {}".format(np.log10(true_x[i-1]),i))
fig.legend(["mean", "median", "Emperical SFS"], loc="lower center", ncol=4)
plt.savefig('ppc_check_hist_mis_36_coefficients_round2_blur_2_hs.png')
plt.close()



In [197]:
plt.ioff()
plt.tight_layout()

fig = plt.figure(figsize=(30,30))
fig.subplots_adjust(hspace=0.6, wspace=0.6)
for i in range(1, 25):
    ax = fig.add_subplot(5, 5, i)
    sns.histplot(np.log10(new_predicted_fs2[:,i-1]))
    plt.axvline(x=np.mean(np.log10(new_predicted_fs2[:,i-1])), color='m', label="mean")
    plt.axvline(x=np.median(np.log10(new_predicted_fs2[:,i-1])), color='k', label="median")
    ax.axline((np.log10(true_x[i-1]), 1), (np.log10(true_x[i-1]),100), marker='+', c='r', label="Emperical SFS")
    plt.title("Emperical SFS: 10^{:.3f} at bin: {}".format(np.log10(true_x[i-1]),i))
fig.legend(["mean", "median", "Emperical SFS"], loc="lower center", ncol=4)
plt.savefig('ppc_check_hist_lof_36_coefficients_round2_blur_2_s.png')
plt.close()



In [48]:
mean_predicted = np.mean(new_predicted_fs[:],axis=0)
print(f"Without pmsid: {mean_predicted[:10]}")
print(true_x[:10])

Without pmsid: [846931.7    197617.72    89259.19    47550.04    27361.467   16637.205
  10606.516    7071.227    4928.155    3590.4414]
tensor([[1410237.,  365768.,  168386.,  ...,       0.,       0.,       0.]],
       device='cuda:0')


In [108]:
x = np.arange(0,mean_predicted[0:200].shape[0])
print("Shape of frequency bins on x-axis for plotting {}".format(x.shape[0]))

Shape of frequency bins on x-axis for plotting 200


In [49]:
fig = plt.figure(figsize=(8,8))
sns.scatterplot(x=x,y=np.log10(mean_predicted[:200]+1), label="Predicted")
sns.scatterplot(x=x,y=np.log10(true_x.cpu().numpy()[0,:200]+1), label="Emperical")
plt.title("Mean of posterior predicted SFS vs True SFS")
plt.ylabel("Log Scaled Allele Frequency")
plt.xlabel("Frequency Bin")
#fig.legend(["Predicted", "Emperical"], loc="lower center", ncol=2)
plt.savefig('ppc_scatter_mis_classifier_check.png')
plt.close()


In [338]:
fig = plt.figure(figsize=(8,8))
sns.scatterplot(x=x,y=np.log10(mean_predicted2+1), label="Predicted")
sns.scatterplot(x=x,y=np.log10(true_x+1), label="Emperical")
plt.title("Mean of posterior predicted SFS vs True SFS")
plt.ylabel("Log Scaled Allele Frequency")
plt.xlabel("Frequency Bin")
#fig.legend(["Predicted", "Emperical"], loc="lower center", ncol=2)
plt.savefig('ppc_scatter_mis_36_coefficients_round1.png')
plt.close()


In [115]:
obs_samples_lof = proposal_lof.sample((20000,))
obs_samples_mis = proposal_mis.sample((20000,))

print("Print missense shape: {}".format(obs_samples_mis.shape))
print("Print lof shape: {}".format(obs_samples_lof.shape))



obs_samples_lof_reshaped = obs_samples_lof.reshape(-1)
obs_samples_mis_reshaped = obs_samples_mis.reshape(-1)


Print missense shape: torch.Size([20000, 60])
Print lof shape: torch.Size([20000, 60])


In [70]:
# For calculating the mean of all samples
mean_samps = []
for i in range(0,1000):
    temp = proposal.sample((5000,))
    mean_samps.append(temp.mean(dim=0))

    

In [50]:
mean_samps = torch.cat(mean_samps)


In [112]:
prior_samps =last_posterior._prior.sample((2000,)).view(-1).cpu().numpy()
print("The prior shape should be the same as the proposals: {}".format(last_posterior._prior.sample((10,)).shape))

The prior shape should be the same as the proposals: torch.Size([10, 60])


In [98]:
plt.close()
fig = plt.figure(figsize=(8,8))
plt.xscale('log')
sns.kdeplot(10**samps, label="DFE", c='r')
plt.title("Kernel Density Estimation Inferred Scaled Selection")
plt.ylabel("Density")
plt.xlabel("Log of Absolute Scaled Selection Coefficient")
fig.legend(["DFE", "Initial Proposal"], loc="lower center", ncol=2)
plt.savefig('ppc_nfe_lof_selectiion.png')
plt.close()

    

In [53]:
plt.close()
fig = plt.figure(figsize=(8,8))
plt.xscale('log')
sns.histplot(10**mean_samps.cpu().numpy())
plt.title("Histogram Plot of Inferred Scaled Selection -- Averaged out over multiple samples")
plt.ylabel("Density")
plt.xlabel("Log of Absolute Scaled Selection Coefficient")
plt.savefig('ppc_nfe_mis_selectiion_classifier.png')
plt.close()

In [117]:
plt.close()
fig = plt.figure(figsize=(8,8))
plt.xscale('log')
sns.histplot(10**obs_samples_mis_reshaped.cpu().numpy())
plt.title("Histogram Plot Inferred of Scaled Selection")
plt.ylabel("Density")
plt.xlabel("Log of Absolute Scaled Selection Coefficient")
plt.savefig('NFE_MIS_DFE.png')
plt.close()

In [118]:
dfe_mis= obs_samples_mis_reshaped.squeeze().cpu().numpy()

In [122]:
dfe_lof= obs_samples_lof_reshaped.squeeze().cpu().numpy()

In [124]:
bins = [-7, -6, -5, -4, -3, -2, -1, 0, 4.0]
for s0, s1 in zip(bins[:-1], bins[1:]):
    the_dat=np.extract((s0 <= dfe_mis) & (dfe_mis < s1), dfe_mis)
    prop = the_dat.shape[0]/obs_samples_mis_reshaped.shape[0]
    print(f"{10**s0} <= |s| < {10**s1}: {prop:.7f}")
    if s1 == bins[-1]:
        the_dat=np.extract(dfe_mis > s1, dfe_mis)
        prop = the_dat.shape[0]/500000.0
        print(f"|s| > {10**s1}: {prop:.7f}")

1e-07 <= |s| < 1e-06: 0.1509767
1e-06 <= |s| < 1e-05: 0.1437117
1e-05 <= |s| < 0.0001: 0.1329875
0.0001 <= |s| < 0.001: 0.1224242
0.001 <= |s| < 0.01: 0.1104300
0.01 <= |s| < 0.1: 0.0970933
0.1 <= |s| < 1: 0.0852250
1 <= |s| < 10000.0: 0.0000000
|s| > 10000.0: 0.0000000


In [125]:
bins = [-7, -6, -5, -4, -3, -2, -1, 0, 4.0]
for s0, s1 in zip(bins[:-1], bins[1:]):
    the_dat=np.extract((s0 <= dfe_mis) & (dfe_lof < s1), dfe_lof)
    prop = the_dat.shape[0]/obs_samples_lof_reshaped.shape[0]
    print(f"{10**s0} <= |s| < {10**s1}: {prop:.7f}")
    if s1 == bins[-1]:
        the_dat=np.extract(dfe_lof > s1, dfe_lof)
        prop = the_dat.shape[0]/500000.0
        print(f"|s| > {10**s1}: {prop:.7f}")

1e-07 <= |s| < 1e-06: 0.1864025
1e-06 <= |s| < 1e-05: 0.2308900
1e-05 <= |s| < 0.0001: 0.2467042
0.0001 <= |s| < 0.001: 0.2379450
0.001 <= |s| < 0.01: 0.2056867
0.01 <= |s| < 0.1: 0.1539450
0.1 <= |s| < 1: 0.0852250
1 <= |s| < 10000.0: 0.0000000
|s| > 10000.0: 0.0000000


In [135]:
# Loss between true and predicted in poisson log-liklihood
loss = -1*torch.nn.functional.poisson_nll_loss(torch.log(torch.tensor(mean_predicted2+1)), torch.log(torch.tensor(true_x+1)),log_input=True, full=False, reduction='sum' )

In [248]:
log_predicted = torch.log(torch.tensor(new_predicted_fs).unsqueeze(1))

In [249]:
log_target = torch.log(torch.tensor(true_x+1).repeat(log_predicted.shape[0],1).unsqueeze(1))

In [250]:
loss2 = -1*torch.nn.functional.poisson_nll_loss(log_predicted, log_target, reduction='mean')

In [213]:
print("poisson log-liklihood loss {} vs moments log-liklihood loss for lof {}".format(loss, moments_loss_lof))

poisson log-liklihood loss -4795.529159783006 vs moments log-liklihood loss for lof -237.79096099886482


In [252]:
print("poisson log-liklihood loss over batch {} vs moments log-liklihood loss for lof {}".format(loss2, moments_loss_lof))

poisson log-liklihood loss over batch -47.59079092120809 vs moments log-liklihood loss for lof -237.79096099886482


In [58]:
resid = (mean_predicted - true_x)/np.sqrt(mean_predicted)

In [59]:
fig = plt.figure(figsize=(8,8))
sns.scatterplot(x=x,y=resid, label="Residual")
plt.title("Poisson Residuals")
plt.ylabel("Residuals")
plt.xlabel("Frequency Bin")
#fig.legend(["Predicted", "Emperical"], loc="lower center", ncol=2)
plt.savefig('resid_lof_32_coefficients_round10.png')
plt.close()

In [138]:
log_target[0]

tensor([[6.5309, 5.0752, 4.4188, 4.0943, 3.6376, 3.1781, 3.1781, 2.6391, 2.5649,
         2.7726, 2.7081, 2.7081, 2.7081, 2.7726, 2.3026, 1.0986, 2.3026, 2.3979,
         1.6094, 1.9459, 1.3863, 1.6094, 1.3863, 1.9459, 1.9459, 1.7918, 1.3863,
         1.6094, 1.0986, 1.0986, 1.6094, 1.7918, 2.0794, 1.6094, 1.0986, 1.6094,
         2.0794, 0.6931, 1.0986, 1.3863, 1.3863, 0.6931, 1.0986, 0.6931, 0.0000,
         1.0986, 0.6931, 1.0986, 0.6931, 0.6931, 0.0000, 1.0986, 1.0986, 0.6931,
         1.0986, 1.0986, 0.6931, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6931,
         0.0000, 0.6931, 0.6931, 0.6931, 0.0000, 0.0000, 0.6931, 0.6931, 0.0000,
         0.6931, 0.6931, 1.3863, 0.6931, 0.6931, 0.0000, 0.6931, 0.6931, 0.0000,
         0.0000, 0.0000, 1.3863, 0.6931, 0.6931, 0.6931, 0.0000, 0.6931, 0.6931,
         1.0986, 0.0000, 0.6931, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0986,
         0.0000, 0.0000, 0.0000, 0.6931, 0.0000, 0.6931, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0